In [2]:
from pathlib import Path

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer


def idf():
  file = open(str(Path.home()) + "\Documents\Python\PJ4\C2\documentosNoDuplicate.txt","r", encoding="utf8")
  liststrings = file.readlines()
  liststrings.remove('Nao lidaBem-vindo ao Facebook! Toque aqui para encontrar pessoas que voce conhece e adiciona-las aos amigos.\n')
  liststrings.remove('\n')

  tfIdfVectorizer=TfidfVectorizer(decode_error ='ignore',use_idf=True, lowercase=True)
  tf_idf = tfIdfVectorizer.fit_transform(liststrings)
  dataIdf = pd.DataFrame(tfIdfVectorizer.idf_ , index=tfIdfVectorizer.get_feature_names(), columns=['IDF']).T
  dataTf_Idf = pd.DataFrame(tf_idf.todense(), columns=tfIdfVectorizer.get_feature_names())
  
  
  postagensData = pd.DataFrame(liststrings, columns=['Postagens']).dropna()

  for col in dataIdf.columns:
    if(col != 'Unnamed: 0'):
        dataTf_Idf.loc[dataTf_Idf[col] != 0.0, col] = dataIdf.iloc[0][col]

  return dataTf_Idf.dropna(), postagensData

def ICF():
   postagensRotulado = pd.read_csv('postagensRotulado.csv', sep=';', decimal=',') 
   finalIdfVector = pd.read_csv('idfRotulado.csv', sep=';', decimal=',').drop(['Classification'], axis=1)

   postagens2 = postagensRotulado.where(postagensRotulado["Classification"] == 2).dropna().Postagens
   postagens1 = postagensRotulado.where(postagensRotulado["Classification"] == 1).dropna().Postagens
   postagens0 = postagensRotulado.where(postagensRotulado["Classification"] == 0).dropna().Postagens

   achou = False
   icfVal = []

   for word in finalIdfVector.columns:
      qtdWord = 0
      for post in postagens2:
         if(achou):
            break
         for word2 in post.split():
            if(achou):
               break
            if(word == word2):
               qtdWord += 1
               achou = True
               break
      achou = False
      for post in postagens1:
         if(achou):
            break
         for word2 in post.split():
            if(achou):
               break
            if(word == word2):
               qtdWord += 1
               achou = True
               break
      achou = False
      for post in postagens0:
         if(achou):
            break
         for word2 in post.split():
            if(achou):
               break
            if(word == word2):
               qtdWord += 1
               achou = True
               break
      if(qtdWord != 0):
         icfVal.append(math.log(1 + 3/qtdWord))
      if(qtdWord == 0):
         icfVal.append(0)

   dataIcf = pd.DataFrame(icfVal , index=finalIdfVector.columns).T   

   for col in dataIcf.columns:
      if(col != 'Unnamed: 0'):
         finalIdfVector.loc[finalIdfVector[col] != 0.0, col] = dataIcf.iloc[0][col] 

   finalIdfVector['Classification'] =   postagensRotulado.Classification 
   finalIdfVector.to_csv('ICF.csv', sep=';', decimal=',', float_format='%.3f') 

def RfIdf():
  file = open(str(Path.home()) + "\Documents\Python\PJ4\C2\documentosNoDuplicate.txt","r", encoding="utf8")
  liststrings = file.readlines()
  liststrings.remove('Nao lidaBem-vindo ao Facebook! Toque aqui para encontrar pessoas que voce conhece e adiciona-las aos amigos.\n')
  liststrings.remove('\n')

  tfIdfVectorizer=TfidfVectorizer(decode_error ='ignore',use_idf=True, lowercase=True)
  tf_idf = tfIdfVectorizer.fit_transform(liststrings)
  dataTf_Idf = pd.DataFrame(tf_idf.todense(), columns=tfIdfVectorizer.get_feature_names())
  #dataRF = pd.DataFrame({ "Words": [], "RF": []})
  
  qtdTotal = 0
  x = 0
  rfidfVal = []
  
  for post in liststrings:
     qtdTotal += len(post.split())

  for word in tfIdfVectorizer.get_feature_names():
     qtdWord = 0
     for post in liststrings:
        for word2 in post.split():
          if(word == word2):
             qtdWord += 1
     rfidfVal.append(qtdWord/qtdTotal * tfIdfVectorizer.idf_[x] )
     x += 1
     #dataRF = dataRF.append({ "Words": word, "RF": qtdWord/qtdTotal}, ignore_index=True)
  
   
  dataRfIdf = pd.DataFrame(rfidfVal , index=tfIdfVectorizer.get_feature_names(), columns=['IDF']).T    
  
  for col in dataRfIdf.columns:
    if(col != 'Unnamed: 0'):
        dataTf_Idf.loc[dataTf_Idf[col] != 0.0, col] = dataRfIdf.iloc[0][col]

  return dataTf_Idf.dropna()




idfvector, postagensData = idf()

kmeans = KMeans(n_clusters=3, random_state=0).fit(idfvector)

idfvector['Classification'] = kmeans.labels_
postagensData['Classification'] = kmeans.labels_

def aleatorio(x):
  postagemAleatoria = x[x.Classification == 0].sample(n = 5, random_state= 0, replace = True);
  postagemAleatoria = postagemAleatoria.append(x[x.Classification == 1].sample(n = 5, random_state= 0, replace = True));
  postagemAleatoria = postagemAleatoria.append(x[x.Classification == 2].sample(n = 5, random_state= 0, replace = True));
  return postagemAleatoria

y = aleatorio(postagensData)

finalIdfVector = idfvector[idfvector.Classification == 1]
idfvector = idfvector[idfvector.Classification != 1]
idfvector = idfvector.drop(columns=['Classification'])

finalpostagensData = postagensData[postagensData.Classification == 1]
postagensData = postagensData[postagensData.Classification != 1]
postagensData = postagensData.drop(columns=['Classification'])

kmeans = KMeans(n_clusters=3, random_state=0).fit(idfvector)

idfvector['Classification'] = kmeans.labels_
postagensData['Classification'] = kmeans.labels_

y = aleatorio(postagensData)

idfvector1 = idfvector[idfvector.Classification == 1].drop(columns=['Classification'])
idfvector2 = idfvector[idfvector.Classification == 2].drop(columns=['Classification'])
idfvector0 = idfvector[idfvector.Classification == 0].drop(columns=['Classification'])

postagensData1 = postagensData[postagensData.Classification == 1].drop(columns=['Classification'])
postagensData2 = postagensData[postagensData.Classification == 2].drop(columns=['Classification'])
postagensData0 = postagensData[postagensData.Classification == 0].drop(columns=['Classification'])

kmeans = KMeans(n_clusters=3, random_state=0).fit(idfvector1)

idfvector1['Classification'] = kmeans.labels_
postagensData1['Classification'] = kmeans.labels_

y = aleatorio(postagensData1)

lines =  idfvector1[idfvector1.Classification == 0]
lines['Classification'] = lines['Classification'].replace(0,1)
finalIdfVector = finalIdfVector.append(lines)
idfvector1 = idfvector1[idfvector1.Classification != 0]
idfvector1 = idfvector1.drop(columns=['Classification'])

lines =  postagensData1[postagensData1.Classification == 0]
lines['Classification'] = lines['Classification'].replace(0,1)
finalpostagensData = finalpostagensData.append(lines)
postagensData1 = postagensData1[postagensData1.Classification != 0]
postagensData1 = postagensData1.drop(columns=['Classification'])

kmeans = KMeans(n_clusters=3, random_state=0).fit(idfvector1)

idfvector1['Classification'] = kmeans.labels_
postagensData1['Classification'] = kmeans.labels_

y = aleatorio(postagensData1)

finalIdfVector = finalIdfVector.append(idfvector1[idfvector1.Classification == 2])
idfvector1 = idfvector1[idfvector1.Classification != 2]
idfvector1 = idfvector1.drop(columns=['Classification'])

finalpostagensData = finalpostagensData.append(postagensData1[postagensData1.Classification == 2])
postagensData1 = postagensData1[postagensData1.Classification != 2]
postagensData1 = postagensData1.drop(columns=['Classification'])

kmeans = KMeans(n_clusters=3, random_state=0).fit(idfvector1)

idfvector1['Classification'] = kmeans.labels_
postagensData1['Classification'] = kmeans.labels_

y = aleatorio(postagensData1)

lines =  idfvector1[idfvector1.Classification == 2]
lines['Classification'] = lines['Classification'].replace(2,0)
finalIdfVector = finalIdfVector.append(lines)

lines =  idfvector1[idfvector1.Classification != 2]
lines['Classification'] = lines['Classification'].replace(0,1)
finalIdfVector = finalIdfVector.append(lines)

lines =  postagensData1[postagensData1.Classification == 2]
lines['Classification'] = lines['Classification'].replace(2,0)
finalpostagensData = finalpostagensData.append(lines)

lines =  postagensData1[postagensData1.Classification != 2]
lines['Classification'] = lines['Classification'].replace(0,1)
finalpostagensData = finalpostagensData.append(lines)

kmeans = KMeans(n_clusters=3, random_state=0).fit(idfvector2)

idfvector2['Classification'] = kmeans.labels_
postagensData2['Classification'] = kmeans.labels_

y = aleatorio(postagensData2)

lines =  idfvector2[idfvector2.Classification != 2]
lines['Classification'] = lines['Classification'].replace(0,1)
finalIdfVector = finalIdfVector.append(lines)
idfvector2 = idfvector2[idfvector2.Classification == 2]
idfvector2 = idfvector2.drop(columns=['Classification'])

lines =  postagensData2[postagensData2.Classification != 2]
lines['Classification'] = lines['Classification'].replace(0,1)
finalpostagensData = finalpostagensData.append(lines)
postagensData2 = postagensData2[postagensData2.Classification == 2]
postagensData2 = postagensData2.drop(columns=['Classification'])

kmeans = KMeans(n_clusters=3, random_state=0).fit(idfvector2)

idfvector2['Classification'] = kmeans.labels_
postagensData2['Classification'] = kmeans.labels_

y = aleatorio(postagensData2)

finalIdfVector = finalIdfVector.append(idfvector2[idfvector2.Classification == 1])
idfvector2 = idfvector2[idfvector2.Classification != 1]
idfvector2 = idfvector2.drop(columns=['Classification'])

finalpostagensData = finalpostagensData.append(postagensData2[postagensData2.Classification == 1])
postagensData2 = postagensData2[postagensData2.Classification != 1]
postagensData2 = postagensData2.drop(columns=['Classification'])

kmeans = KMeans(n_clusters=3, random_state=0).fit(idfvector2)

idfvector2['Classification'] = kmeans.labels_
postagensData2['Classification'] = kmeans.labels_

y = aleatorio(postagensData2)

lines =  idfvector2[idfvector2.Classification == 2]
lines['Classification'] = lines['Classification'].replace(2,0)
finalIdfVector = finalIdfVector.append(lines)

lines =  idfvector2[idfvector2.Classification != 2]
lines['Classification'] = lines['Classification'].replace(0,1)
finalIdfVector = finalIdfVector.append(lines)

lines =  postagensData2[postagensData2.Classification == 2]
lines['Classification'] = lines['Classification'].replace(2,0)
finalpostagensData = finalpostagensData.append(lines)

lines =  postagensData2[postagensData2.Classification != 2]
lines['Classification'] = lines['Classification'].replace(0,1)
finalpostagensData = finalpostagensData.append(lines)

kmeans = KMeans(n_clusters=3, random_state=0).fit(idfvector0)

idfvector0['Classification'] = kmeans.labels_
postagensData0['Classification'] = kmeans.labels_

y = aleatorio(postagensData0)

finalIdfVector = finalIdfVector.append(idfvector0)

finalpostagensData = finalpostagensData.append(postagensData0)

finalIdfVector.to_csv('idfRotulado.csv', sep=';', decimal=',', float_format='%.3f') 
finalpostagensData.to_csv('postagensRotulado.csv', sep=';', decimal=',', float_format='%.3f') 
ICF()




c:\Users\breno\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\breno\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\breno\AppData\Local\Temp\ipykernel_14352\882793358.py:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  postagemAleatoria = postagemAleatoria.append(x[x.Classification == 1].sample(n = 5, random_state= 0, replace = True));
C:\Users\breno\AppData\Local\Temp\ipykernel_14352\882793358.py:39: FutureWarning: The frame.append method

In [21]:
from pathlib import Path
import math
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

def RfIdf():
  file = open(str(Path.home()) + "\Documents\Python\PJ4\C2\documentosNoDuplicate.txt","r", encoding="utf8")
  liststrings = file.readlines()
  liststrings.remove('Nao lidaBem-vindo ao Facebook! Toque aqui para encontrar pessoas que voce conhece e adiciona-las aos amigos.\n')
  liststrings.remove('\n')

  postagensRotulado = pd.read_csv('postagensRotulado.csv', sep=';', decimal=',', float_format='%.3f') 

  tfIdfVectorizer=TfidfVectorizer(decode_error ='ignore',use_idf=True, lowercase=True)
  tf_idf = tfIdfVectorizer.fit_transform(liststrings)
  dataTf_Idf = pd.DataFrame(tf_idf.todense(), columns=tfIdfVectorizer.get_feature_names())
  #dataRF = pd.DataFrame({ "Words": [], "RF": []})
  
  qtdTotal = 0
  x = 0
  rfidfVal = []
  
  for post in liststrings:
     qtdTotal += len(post.split())

  for word in tfIdfVectorizer.get_feature_names():
     qtdWord = 0
     for post in liststrings:
        for word2 in post.split():
          if(word == word2):
             qtdWord += 1
     rfidfVal.append(qtdWord/qtdTotal * tfIdfVectorizer.idf_[x] )
     x += 1
     #dataRF = dataRF.append({ "Words": word, "RF": qtdWord/qtdTotal}, ignore_index=True)
  
   
  dataRfIdf = pd.DataFrame(rfidfVal , index=tfIdfVectorizer.get_feature_names(), columns=['IDF']).T    
  
  for col in dataRfIdf.columns:
    if(col != 'Unnamed: 0'):
        dataTf_Idf.loc[dataTf_Idf[col] != 0.0, col] = dataRfIdf.iloc[0][col]

  return dataTf_Idf.dropna()
  

def ICF():
   postagensRotulado = pd.read_csv('postagensRotulado.csv', sep=';', decimal=',') 
   finalIdfVector = pd.read_csv('idfRotulado.csv', sep=';', decimal=',').drop(['Classification'], axis=1)

   postagens2 = postagensRotulado.where(postagensRotulado["Classification"] == 2).dropna().Postagens
   postagens1 = postagensRotulado.where(postagensRotulado["Classification"] == 1).dropna().Postagens
   postagens0 = postagensRotulado.where(postagensRotulado["Classification"] == 0).dropna().Postagens

   achou = False
   icfVal = []

   for word in finalIdfVector.columns:
      qtdWord = 0
      for post in postagens2:
         if(achou):
            break
         for word2 in post.split():
            if(achou):
               break
            if(word == word2):
               qtdWord += 1
               achou = True
               break
      achou = False
      for post in postagens1:
         if(achou):
            break
         for word2 in post.split():
            if(achou):
               break
            if(word == word2):
               qtdWord += 1
               achou = True
               break
      achou = False
      for post in postagens0:
         if(achou):
            break
         for word2 in post.split():
            if(achou):
               break
            if(word == word2):
               qtdWord += 1
               achou = True
               break
      if(qtdWord != 0):
         icfVal.append(math.log(1 + 3/qtdWord))
      if(qtdWord == 0):
         icfVal.append(0)

   dataIcf = pd.DataFrame(icfVal , index=finalIdfVector.columns).T   

   for col in dataIcf.columns:
      if(col != 'Unnamed: 0'):
         finalIdfVector.loc[finalIdfVector[col] != 0.0, col] = dataIcf.iloc[0][col] 

   finalIdfVector['Classification'] =   postagensRotulado.Classification 
   finalIdfVector.to_csv('ICF.csv', sep=';', decimal=',', float_format='%.3f') 
          


In [ ]:
from pathlib import Path
import math

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

def ICF():
  file = open(str(Path.home()) + "\Documents\Python\PJ4\C2\documentosNoDuplicate.txt","r", encoding="utf8")
  liststrings = file.readlines()
  liststrings.remove('Nao lidaBem-vindo ao Facebook! Toque aqui para encontrar pessoas que voce conhece e adiciona-las aos amigos.\n')
  liststrings.remove('\n')

  tfIdfVectorizer=TfidfVectorizer(decode_error ='ignore',use_idf=True, lowercase=True)
  tf_idf = tfIdfVectorizer.fit_transform(liststrings)
  dataTf_Idf = pd.DataFrame(tf_idf.todense(), columns=tfIdfVectorizer.get_feature_names())
  
  x = 0
  icfVal = []
  
  for post in liststrings:
     qtdTotal += len(post.split())

  for word in tfIdfVectorizer.get_feature_names():
     qtdWord = 0
     for post in liststrings:
        for word2 in post.split():
          if(word == word2):
             qtdWord += 1
     rfidfVal.append(qtdWord/qtdTotal * tfIdfVectorizer.idf_[x] )
     x += 1
     #dataRF = dataRF.append({ "Words": word, "RF": qtdWord/qtdTotal}, ignore_index=True)
  
   
  dataRfIdf = pd.DataFrame(rfidfVal , index=tfIdfVectorizer.get_feature_names(), columns=['IDF']).T    
  
  for col in dataRfIdf.columns:
    if(col != 'Unnamed: 0'):
        dataTf_Idf.loc[dataTf_Idf[col] != 0.0, col] = dataRfIdf.iloc[0][col]

  return dataTf_Idf.dropna()
  

teste = RfIdf()